In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.layers import AveragePooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import argparse
import os
import tensorflow as tf
sns.set_style('whitegrid')
from glob import glob


Using TensorFlow backend.


In [2]:
df_NIH = pd.read_csv(r'C:\Users\Zijian\OneDrive - Cranfield University\Group project\NIH\Data_Entry_2017.csv')
# only keep the illnesss label and image names

df_NIH = df_NIH[['Image Index', 'Finding Labels']]
# create a column to store the full path of images for later reading
# You should change the below path when running at your computer

my_glob = glob('C:/Users/Zijian/OneDrive - Cranfield University/Group project/NIH/images/images/*.png')
print('Number of Observations: ', len(my_glob))
full_img_paths = {os.path.basename(x): x for x in my_glob}

df_NIH['path'] = df_NIH['Image Index'].map(full_img_paths.get)
# extract four common illness from NIH datasets

df_NIH_effusion = df_NIH[df_NIH['Finding Labels'].isin(['Effusion'])]
df_NIH_atelectasis = df_NIH[df_NIH['Finding Labels'].isin(['Atelectasis'])]
df_NIH_infiltration = df_NIH[df_NIH['Finding Labels'].isin(['Infiltration'])]
df_NIH_pneumonia = df_NIH[df_NIH['Finding Labels'].isin(['Pneumonia'])]
df_NIH_none = df_NIH[df_NIH['Finding Labels'].isin(['No Finding'])]



Number of Observations:  112120


In [3]:
# only keep 20000 health images

df_NIH_none = df_NIH_none.sample(n=20000)
# add 3800 pneumonia images from other datasets

glob2 = glob('C:/Users/Zijian/Cranfield University/Muhammad Hasan Ali, Hasan - GroupProject/Research/PNEUMONIA/*.jpeg')
df_Phe = pd.DataFrame(glob2, columns=['path'])

df_Phe['Finding Labels'] = 'Pneumonia'
# concat the NIH pneumonia and other pneumonia images together

df_NIH_pneumonia = pd.concat([df_NIH_pneumonia, df_Phe])
print('effusion :', len(df_NIH_effusion))
print('atelectasis :', len(df_NIH_atelectasis))
print('infiltration :', len(df_NIH_infiltration))
print('pneumonia :', len(df_NIH_pneumonia))
print('health :', len(df_NIH_none))


effusion : 3955
atelectasis : 4215
infiltration : 9547
pneumonia : 4197
health : 20000


In [4]:
# only keep 20000 health images

df_NIH_none = df_NIH_none.sample(n=40000)
# add 3800 pneumonia images from other datasets

glob2 = glob('C:/Users/Zijian/Cranfield University/Muhammad Hasan Ali, Hasan - GroupProject/Research/PNEUMONIA/*.jpeg')
df_Phe = pd.DataFrame(glob2, columns=['path'])

df_Phe['Finding Labels'] = 'Pneumonia'
# concat the NIH pneumonia and other pneumonia images together

df_NIH_pneumonia = pd.concat([df_NIH_pneumonia, df_Phe])
print('effusion :', len(df_NIH_effusion))
print('atelectasis :', len(df_NIH_atelectasis))
print('infiltration :', len(df_NIH_infiltration))
print('pneumonia :', len(df_NIH_pneumonia))
print('health :', len(df_NIH_none))


ValueError: Cannot take a larger sample than population when 'replace=False'

In [5]:
# only keep 20000 health images

df_NIH_none = df_NIH_none.sample(n=30000)
# add 3800 pneumonia images from other datasets

glob2 = glob('C:/Users/Zijian/Cranfield University/Muhammad Hasan Ali, Hasan - GroupProject/Research/PNEUMONIA/*.jpeg')
df_Phe = pd.DataFrame(glob2, columns=['path'])

df_Phe['Finding Labels'] = 'Pneumonia'
# concat the NIH pneumonia and other pneumonia images together

df_NIH_pneumonia = pd.concat([df_NIH_pneumonia, df_Phe])
print('effusion :', len(df_NIH_effusion))
print('atelectasis :', len(df_NIH_atelectasis))
print('infiltration :', len(df_NIH_infiltration))
print('pneumonia :', len(df_NIH_pneumonia))
print('health :', len(df_NIH_none))


ValueError: Cannot take a larger sample than population when 'replace=False'

In [6]:
# extract four common illness from NIH datasets

df_NIH_effusion = df_NIH[df_NIH['Finding Labels'].isin(['Effusion'])]
df_NIH_atelectasis = df_NIH[df_NIH['Finding Labels'].isin(['Atelectasis'])]
df_NIH_infiltration = df_NIH[df_NIH['Finding Labels'].isin(['Infiltration'])]
df_NIH_pneumonia = df_NIH[df_NIH['Finding Labels'].isin(['Pneumonia'])]
df_NIH_none = df_NIH[df_NIH['Finding Labels'].isin(['No Finding'])]


In [7]:
# only keep 20000 health images

df_NIH_none = df_NIH_none.sample(n=40000)
# add 3800 pneumonia images from other datasets

glob2 = glob('C:/Users/Zijian/Cranfield University/Muhammad Hasan Ali, Hasan - GroupProject/Research/PNEUMONIA/*.jpeg')
df_Phe = pd.DataFrame(glob2, columns=['path'])

df_Phe['Finding Labels'] = 'Pneumonia'
# concat the NIH pneumonia and other pneumonia images together

df_NIH_pneumonia = pd.concat([df_NIH_pneumonia, df_Phe])
print('effusion :', len(df_NIH_effusion))
print('atelectasis :', len(df_NIH_atelectasis))
print('infiltration :', len(df_NIH_infiltration))
print('pneumonia :', len(df_NIH_pneumonia))
print('health :', len(df_NIH_none))


effusion : 3955
atelectasis : 4215
infiltration : 9547
pneumonia : 4197
health : 40000


In [8]:
# concat the five dataframe together

xray_data = pd.concat([df_NIH_effusion, df_NIH_atelectasis, 
	df_NIH_infiltration, df_NIH_pneumonia, df_NIH_none])


In [9]:
# split data into train, test, validation

train_set, valid_set = train_test_split(xray_data, test_size = 0.02, random_state = 42)
train_set, test_set = train_test_split(train_set, test_size = 0.2, random_state = 8545)



In [10]:
print('training set: ', len(train_set))
print('test set: ', len(test_set))
print('validation set: ', len(valid_set))
print('full data set: ', len(xray_data))


training set:  48540
test set:  12135
validation set:  1239
full data set:  61914


In [11]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)


In [12]:
image_size = (128, 128) # image re-sizing target
all_classes = ['Effusion', 'Atelectasis', 'Infiltration', 'Pneumonia', 'No Finding']


In [13]:
train_generator = train_datagen.flow_from_dataframe(
	dataframe=train_set,
	directory=None,
	x_col='path', 
	y_col = 'Finding Labels',
	target_size=image_size,
	color_mode='rgb',
	batch_size=32,
	classes=all_classes,
	class_mode='categorical'
)

validation_generator = train_datagen.flow_from_dataframe(
	dataframe=test_set,
	directory=None,
	x_col='path', 
	y_col = 'Finding Labels',
	target_size=image_size,
	color_mode='rgb',
	batch_size=32,
	classes=all_classes,
	class_mode='categorical'	
)

test_X, test_Y = next(test_datagen.flow_from_dataframe(
	dataframe=valid_set,
	directory=None,
	x_col='path', 
	y_col = 'Finding Labels',
	target_size=image_size,
	color_mode='rgb',
	batch_size= 1200,
	classes=all_classes,
	class_mode='categorical'
	))
# load the VGG16 network, ensuring the head FC layer sets are left off

baseModel = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(128, 128, 3)))
# construct the head of the model that will be placed on top of the
# the base model

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)

headModel = Dense(5, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)

model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process

for layer in baseModel.layers:
	layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
print("[INFO] training head...")
EPOCHS = 30

H = model.fit_generator(
        train_generator,
        steps_per_epoch=20,
        epochs=EPOCHS,
        validation_data=validation_generator,
        validation_steps=20)
# make predictions on the testing set

print("[INFO] evaluating network...")

predIdxs = model.predict(test_X)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability

predIdxs2 = np.argmax(predIdxs, axis=1)
print(classification_report(test_Y.argmax(axis=1), predIdxs2))


Found 48540 validated image filenames belonging to 5 classes.
Found 12135 validated image filenames belonging to 5 classes.
Found 1239 validated image filenames belonging to 5 classes.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2

In [14]:
test_Y

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [15]:
test_Y[3]

array([0., 0., 0., 1., 0.], dtype=float32)

In [16]:
train_generator.class_indices

{'Atelectasis': 0,
 'Effusion': 1,
 'Infiltration': 2,
 'No Finding': 3,
 'Pneumonia': 4}

In [17]:
print(train_generator.class_indices)


{'Atelectasis': 0, 'Effusion': 1, 'Infiltration': 2, 'No Finding': 3, 'Pneumonia': 4}


In [18]:
test_Y

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [19]:
predIdxs

array([[0.07303635, 0.06443305, 0.13183169, 0.7258809 , 0.00481791],
       [0.11695655, 0.12222102, 0.19152321, 0.5372569 , 0.03204232],
       [0.05684813, 0.04849407, 0.13311172, 0.7449899 , 0.01655628],
       ...,
       [0.06710953, 0.06047219, 0.16114014, 0.698648  , 0.01263022],
       [0.02398249, 0.021471  , 0.06921637, 0.28923643, 0.5960938 ],
       [0.04983872, 0.0395053 , 0.12436139, 0.7774179 , 0.00887675]],
      dtype=float32)

In [20]:
print("score: ", model.evaluate(test_X, test_Y, verbose=1)) 


1200/1200 [==============================] - 7s 5ms/step
score:  [0.9499785645802816, 0.6866666674613953]


In [21]:
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


NameError: name 'scores' is not defined

In [22]:
score = model.evaluate(test_X, test_Y, verbose=1)
print('Validation loss:', scores[0])
print('Validation accuracy:', scores[1])


1200/1200 [==============================] - 6s 5ms/step


NameError: name 'scores' is not defined

In [23]:
print(classification_report(test_Y.argmax(axis=1), predIdxs2))


precision    recall  f1-score   support

           0       0.00      0.00      0.00        72
           1       0.00      0.00      0.00        81
           2       0.00      0.00      0.00       198
           3       0.67      1.00      0.80       771
           4       0.96      0.69      0.81        78

    accuracy                           0.69      1200
   macro avg       0.33      0.34      0.32      1200
weighted avg       0.50      0.69      0.57      1200



In [24]:
print(train_generator.class_indices)


{'Atelectasis': 0, 'Effusion': 1, 'Infiltration': 2, 'No Finding': 3, 'Pneumonia': 4}


In [25]:
scores = model.evaluate(test_X, test_Y, verbose=1)
print('Validation loss:', scores[0])
print('Validation accuracy:', scores[1])


1200/1200 [==============================] - 6s 5ms/step
Validation loss: 0.9499785645802816
Validation accuracy: 0.6866666674613953


In [26]:
model.compile(optimizer=Adam(learning_rate=0.5e-3), 
	loss='categorical_crossentropy', 
	metrics=['accuracy'])

model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0   

In [28]:
save_dir = os.path.join(os.getcwd(), 'saved_models')


In [29]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

filepath = os.path.join(save_dir, 'VGG16_train')


In [31]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau


In [33]:
# Prepare callbacks for model saving and for learning rate adjustment.

checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer]
